In [1]:
from __future__ import annotations

import argparse
import os
import pathlib
import subprocess
import sys
from typing import Callable, Union

import dlib
import huggingface_hub
import numpy as np
import PIL.Image
import torch
import torch.nn as nn
import torchvision.transforms as T


/home/ishant/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from mapper.datasets.latents_dataset_inference import LatentsDatasetInference
from mapper.hairclip_mapper import HairCLIPMapper

In [3]:
def load_hairclip() -> nn.Module:
        ckpt_path = huggingface_hub.hf_hub_download('public-data/HairCLIP',
                                                    'hairclip.pt')
        ckpt = torch.load(ckpt_path, map_location='cpu')
        opts = ckpt['opts']
        opts['device'] = "cuda"
        opts['checkpoint_path'] = ckpt_path
        opts['editing_type'] = 'both'
        opts['input_type'] = 'text'
        opts['hairstyle_description'] = '/home/ishant/Desktop/test2/HairCLIP/mapper/hairstyle_list.txt'
        opts['color_description'] = 'red'
        opts = argparse.Namespace(**opts)
        model = HairCLIPMapper(opts)
        model.to("cuda")
        model.eval()
        return model
hairclip = load_hairclip() 

Loading from checkpoint: /home/ishant/.cache/huggingface/hub/models--public-data--HairCLIP/snapshots/fead3e4468e56e3e32df67bee20a3c141620c605/hairclip.pt


In [4]:
def denormalize(tensor: torch.Tensor) -> torch.Tensor:
        return torch.clamp((tensor + 1) / 2 * 255, 0, 255).to(torch.uint8)

def postprocess(tensor: torch.Tensor) -> np.ndarray:
        tensor = denormalize(tensor)
        return tensor.cpu().numpy().transpose(1, 2, 0)

In [6]:

def generate(editing_type: str, hairstyle_index: int,
             color_description: str, latent: torch.Tensor) -> np.ndarray:

    opts = hairclip.opts
    opts.editing_type = editing_type
    opts.color_description = color_description
    if editing_type == 'color':
        hairstyle_index = 0
    device = torch.device(opts.device)
    dataset = LatentsDatasetInference(latents=latent.unsqueeze(0).cpu(),
                                      opts=opts)
    w, hairstyle_text_inputs_list, color_text_inputs_list = dataset[0][:3]
    w = w.unsqueeze(0).to(device)
    hairstyle_text_inputs = hairstyle_text_inputs_list[
        hairstyle_index].unsqueeze(0).to(device)
    color_text_inputs = color_text_inputs_list[0].unsqueeze(0).to(device)
    hairstyle_tensor_hairmasked = torch.Tensor([0]).unsqueeze(0).to(device)
    color_tensor_hairmasked = torch.Tensor([0]).unsqueeze(0).to(device)
    print(w.size())
    print(hairstyle_text_inputs.size())
    # print(color_text_inputs.size())
    print(hairstyle_tensor_hairmasked.size())
    
    w_hat = w + 0.1 * hairclip.mapper(
        w,
        hairstyle_text_inputs,
        color_text_inputs,
        hairstyle_tensor_hairmasked,
        color_tensor_hairmasked,
    )
    x_hat, _ = hairclip.decoder(
        [w_hat],
        input_is_latent=True,
        return_latents=True,
        randomize_noise=False,
        truncation=1,
    )
    res = torch.clamp(x_hat[0].detach(), -1, 1)
    res = postprocess(res)
    return res

In [7]:
loaded_latent_vectors = torch.load('inversion_codes/test5.pt')

In [7]:
# print(loaded_latent_vectors.size())

In [32]:


# Call the generate function with the loaded latent vectors
editing_type = "both"
hairstyle_index = 13 # Replace with your desired values
color_description = "blue"
result_image = generate(editing_type, hairstyle_index, color_description, loaded_latent_vectors['latent'])


torch.Size([1, 18, 512])
torch.Size([1, 77])
torch.Size([1, 1])


In [33]:
# loaded_latent_vectors = torch.load('train_faces.pt')
# print(loaded_latent_vectors.size())

In [34]:
result_image.shape

(1024, 1024, 3)